In [ ]:
# KNN
vizinhos = 3

In [ ]:
# Importando as Bibliotecas Básicas
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# Importando a Biblioteca do KNN
from sklearn.neighbors import KNeighborsClassifier
# Importando a Biblioteca de Acurácia, Matriz de Confusão (em quantos jogos entrou em quantos foi green ou red)
from sklearn.metrics import accuracy_score, confusion_matrix
# Importando a Biblioteca SkLearn para padronizar os dados (entre 0 e 1)
from sklearn.preprocessing import StandardScaler
# Importando a Biblioteca SkLearn para dividir os dados entre treinamento e teste
from sklearn.model_selection import train_test_split

In [ ]:
# Importando os DataFrames

df_BR13 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2013.xlsx?raw=true')
df_BR14 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2014.xlsx?raw=true')
df_BR15 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2015.xlsx?raw=true')
df_BR16 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2016.xlsx?raw=true')
df_BR17 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2017.xlsx?raw=true')
df_BR18 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2018.xlsx?raw=true')
df_BR19 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2019.xlsx?raw=true')
df_BR20 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2020.xlsx?raw=true')
df_BR21 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2021.xlsx?raw=true')
df_BR22 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2022.xlsx?raw=true')

# Concatenando os DataFrames
df = [df_BR13,df_BR14,df_BR15,df_BR16,df_BR17,df_BR18,df_BR19,df_BR20,df_BR21]
df = pd.concat(df)
df = df[['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Goals_H','FT_Goals_A']]

In [ ]:
# Criando a Coluna de 0 e 1 para análise do Modelo de Machine Learning
df.loc[(df['FT_Goals_H'] > df['FT_Goals_A']), 'BackHome'] = 1
df.loc[(df['FT_Goals_H'] <= df['FT_Goals_A']), 'BackHome'] = 0

In [ ]:
df

In [ ]:
df1 = df[['FT_Odds_H','FT_Odds_D','FT_Odds_A','BackHome']]

In [ ]:
df1 = df1.dropna()
# convertendo os arrays (matriz)
# definindo qual coluna é a entrada e qual é a saída
df1_x = df1.iloc[:,0:3].values
df1_y = df1.iloc[:,3].values

In [ ]:
df1_x

In [ ]:
df1_y

In [ ]:
# Definições de Parâmetros e dados de entrada e resultado para treinamento
knn_dados = KNeighborsClassifier(n_neighbors=vizinhos, metric="minkowski", p=2)
knn_dados.fit(df1_x, df1_y)

In [ ]:
df2 = df_BR22[['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Goals_H','FT_Goals_A']]

df2.loc[(df2['FT_Goals_H'] > df2['FT_Goals_A']), 'BackHome'] = 1
df2.loc[(df2['FT_Goals_H'] <= df2['FT_Goals_A']), 'BackHome'] = 0

df2 = df2[['FT_Odds_H','FT_Odds_D','FT_Odds_A','BackHome']]

In [ ]:
df2

In [ ]:
df2 = df2.dropna()
# convertendo os arrays (matriz)
# definindo qual coluna é a entrada e qual é a saída
df2_x = df2.iloc[:,0:3].values

# Previsões
previsoes_knn2 = knn_dados.predict(df2_x)

df2["Previsões"] = previsoes_knn2

stake = 1
win_Back = stake * (df2.FT_Odds_H - 1)
lose_Back = -stake

df2.loc[(df2['Previsões'] == 1) & (df2['BackHome'] == 1), 'Profit'] = win_Back
df2.loc[(df2['Previsões'] == 1) & (df2['BackHome'] == 0), 'Profit'] = lose_Back
df2.loc[(df2['Previsões'] == 0) & (df2['BackHome'] == 1), 'Profit'] = 0
df2.loc[(df2['Previsões'] == 0) & (df2['BackHome'] == 0), 'Profit'] = 0

filtro = df2.Previsões == 1
df0 = df2[filtro]

# Ajustando o Índice
df0.reset_index(inplace=True, drop=True)
df0.index = df0.index.set_names(['Nº'])
df0 = df0.rename(index=lambda x: x + 1)

df0['Profit_acu'] = df0.Profit.cumsum()
df0.Profit_acu.plot()
df0.Profit_acu.tail(1)